# Uniwersalna Rura

## Schemat rury
<ol>
    <li>Przed-gotowanie: mart</li>
    <ol>
        <li>Dokumentacja where'ów</li>
        <li>Opcjonalnie: próbkowanie</li>
        
    </ol>
    <li>[Przygotowanie](#k_przygotowanie)</li>
    <ol>
        <li>[Importy](#k_importy) pythonowe</li>
        <li>[Wczytanie](#k_wczytanie) danych</li>
        <li>[Podział zbiorów](#k_podzial): uczący vs walidacyjny vs testowy</li>
        <li>[Rzeźbienie](#k_rzezba) uczącego</li>
        <ol>
            <li>Braki</li>
            <li>Zmienne kategoryczne</li>
            <li>Normalizacja / standaryzacja</li>
            <small>przyspiesza działanie algorytmów, które nie bazują na odległości Euklidesowej (drzewa)
            <br>fit_transform na uczącym, ale na testowym tylko transform
            <br>dla y nie ma znaczenia, jeśli to klasyfikacja; jeśli regresja, to ma znaczenie</small>
            <li>Ficzer endżiniiring</li>
        </ol>
        <li>[Przygot walidacyjnego](#k_przygot_walid)</li>
    </ol>
    <li>[Modelowanie](#k_modelowanie)</li>
    <ol>
        <li>Pierwszy model: [szympans](#k_szympans)</li>
        <li>Wiele [modeli](#k_modele)</li>
        <ol>
            <li>Wiele parametrów</li>
        </ol>
        <li>[Ewaluacja](#k_ewaluacja)</li>
        <li>[Wybór zwycięskiego modelu](#k_wybor_i_test) i oszacowanie jego jakości na testowym (trzeba przygotować)</li>
    </ol>
    <li>[Produkcja](#k_produkcja)</li>
    <ol>
        <li>[Eksport](#k_eksport) - pierwszy ręczny</li>
        <li>Harmonogramy</li>
        <ol>
            <li>Mart</li>
            <li>Skoring: SQL lub import > skrypt zewn. > eksport</li>
            <li>Eksport martu ze skoringiem</li>
            <li>Raportowanie historii modeli</li>
            <li>Ewaluacja - potencjalna zmiana modelu</li>
        </ol>
    </ol>
</ol>

<hr/>

<a id='k_przygotowanie'></a>
## Przygotowanie

http://localhost:8888/notebooks/Projekty/Dejta_Sajens/Machine%20Learning%20A-Z/Part%201%20-%20Data%20Preprocessing/przygot_danych.ipynb

<a id='k_importy'></a>
### Importy pythonowe

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

<a id='k_wczytanie'></a>
### Wczytanie danych

In [6]:
sciezka = 'C:/Users/Szamil/Documents/Projekty/Dejta_Sajens/K01_Titanic/'
plik = 'train.csv'
train_surowy = pd.read_csv(sciezka + plik)

plik = 'test.csv'
test_surowy = pd.read_csv(sciezka + plik)

<a id='k_podzial'></a>
### Podział zbiorów

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

<a id='k_rzezba'></a>
### Rzeźba
Zamknąć w funkcjach, by potem łatwo w ten sam sposób przerobić walidacyjny i testowy.
<br>Czym się zająć:
<ol>
    <li>Rozpoznanie typów, rozkłady</li>
    <li>Braki</li>
    <li>Zmienne kategoryczne</li>
    <li>Korelacje z y</li>
    <li>Normalizacja / standaryzacja</li>
    <small><ul>
        <li>przyspiesza działanie algorytmów, które nie bazują na odległości Euklidesowej (drzewa)</li>
        <li>fit_transform na uczącym, ale na walidacyjnym i testowym tylko transform</li>
        <li>dla y nie ma znaczenia, jeśli to klasyfikacja; jeśli regresja, to ma znaczenie</li>
    </ul></small>
    <li>Ficzer endżiniiring</li>
</ol>
<br>Uwagi:
- .copy()
- .loc(,) lub .iloc(,)

In [7]:
train_robo = train_surowy.copy()
test_robo = test_surowy.copy()
# combined_robo = [train_robo, test_robo]

#### Rozpoznanie

In [18]:
train_robo.shape

(891, 12)

In [14]:
train_robo[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Rozkłady

In [13]:
train_robo.describe()

C:\Users\Szamil\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Przydałoby się coś graficznego.

In [ ]:
# .value_counts([normalize = True])
train["Survived"][train["Sex"] == 'male'].value_counts([normalize = True])

#### Braki

In [15]:
train_robo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [ ]:
# uzupełnienie .fillna()
train["Embarked"] = train["Embarked"].fillna("S")



#### Zmienne kategoryczne

In [ ]:
# ręczne rekodowanie na liczby
train["Embarked"][train["Embarked"] == "S"] = 0 

#### Korelacje z y

#### Normalizacja / standaryzacja

#### Ficzer endżiniiring

<a id='k_przygot_walid'></a>
### Przygot walidacyjnego

In [4]:
# 

<hr/>

<a id='k_modelowanie'></a>
## Modelowanie

<a id='k_szympans'></a>
### Pierwszy model, autorstwa Szympansa

In [2]:
test_robo['Survived'] = 0
test_robo.loc[test_robo['Sex'] == 'female','Survived'] = 1


NameError: name 'test_robo' is not defined

<a id='k_modele'></a>
### Modele na serio
<br>Pokrętła:
<br>**Opcje**
- rebalans

<br>**Modele**
- <a href="http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html">Porównanie na SciKit Learn</a>
- <a href="http://localhost:8888/notebooks/Projekty/Dejta_Sajens/Machine%20Learning%20A-Z/Part%203%20-%20Classification/Section%2020%20-%20Random%20Forest%20Classification/random_forest_moj.ipynb">Notebook z Random Forestem</a>


<br>**Parametry**
- random 60 hyperparameter search

In [ ]:
# przeróbka DataFrame'u na numpy arraye
target = train["Survived"].values

from sklearn import tree

# zasadzenie dżewka
my_tree = tree.DecisionTreeClassifier()
# wersja przystrzyżona
my_tree_two = tree.DecisionTreeClassifier(max_depth = 10, min_samples_split = 5, random_state = 1)
my_tree = my_tree.fit(features, target)

# wyniki
print(my_tree_one.feature_importances_)
print(my_tree_one.score(features, target))


# ====================
# Prediction
# ====================


# Impute the missing value with the median
test.Fare[152] = test["Fare"].median()

# Extract the features from the test set: Pclass, Sex, Age, and Fare.
test_features = test[["Pclass", "Sex", "Age", "Fare"]].values

# Make your prediction using the test set
my_prediction = my_tree_one.predict(test_features)

# Create a data frame with two columns: PassengerId & Survived. Survived contains your predictions
PassengerId =np.array(test["PassengerId"]).astype(int)
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])
print(my_solution)

# Check that your data frame has 418 entries
print(my_solution.shape)

# Write your solution to a csv file with the name my_solution.csv
my_solution.to_csv("my_solution_one.csv", index_label = ["PassengerId"])

# ====================
# Las
# ====================


# Import the `RandomForestClassifier`
from sklearn.ensemble import RandomForestClassifier

# We want the Pclass, Age, Sex, Fare,SibSp, Parch, and Embarked variables
features_forest = train[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values

# Building and fitting my_forest
forest = RandomForestClassifier(max_depth = 10, min_samples_split=2, n_estimators = 100, random_state = 1)
my_forest = forest.fit(features_forest, target)

# Print the score of the fitted random forest
print(my_forest.score(features_forest, target))

# Compute predictions on our test set features then print the length of the prediction vector
test_features = test[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values
pred_forest = my_forest.predict(test_features)
print(len(pred_forest))

<a id='k_ewaluacja'></a>
### Ewaluacja

In [ ]:
# ...

<a id='k_wybor_i_test'></a>
### Wybór zwycięskiego modelu i oszacowanie jego jakości na testowym
Wybrany model wytrenować na obu zbiorach (train+validation), a porównać z trzecim (test).
<br>Trzeba przygotować testowy jak uczący i walidacyjny.
- Uwaga na standaryzację/normalizację: fit_transform na uczącym, ale na walidacyjnym i testowym tylko transform

<hr/>

<a id='k_produkcja'></a>
## Produkcja

<a id='k_eksport'></a>
### Eksport

In [ ]:
do_wysylki = pd.DataFrame({
    'PassengerId':test_robo['PassengerId'],
    'Survived':test_robo['Survived']
})
do_wysylki.to_csv('do_wysylki.csv', index = False)